In [12]:
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
from pyspark.sql.functions import date_format
from pyspark.sql import functions as F
import pandasql as ps
from datetime import date


# load mongo data
input_uri = "mongodb://127.0.0.1/mydbname.mycollection"
output_uri = "mongodb://127.0.0.1/mydbname.mycollection"
#dfs = pdm.read_mongo("mycollection", [], "mongodb://localhost:27017/mydbname")

my_spark = SparkSession\
    .builder\
    .appName("MyApp")\
    .config("spark.mongodb.input.uri", input_uri)\
    .config("spark.mongodb.output.uri", output_uri)\
    .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:2.4.2')\
    .getOrCreate()

df = my_spark.read.format('com.mongodb.spark.sql.DefaultSource').load()

res = df.withColumn('id', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

res.createOrReplaceTempView("temp")
features = my_spark.sql("SELECT id,sessionId,timestamp, userQuery, event,flowName,handled,intentName FROM temp")

q_time = features.withColumn('time', date_format('timestamp', 'HH:mm:ss'))
#time and date added 
date_inclusive = q_time.withColumn('date', date_format('timestamp', 'yyyy-MM-dd'))
userQuery = my_spark.sql("SELECT userQuery,timestamp FROM temp WHERE id IN (SELECT MAX(id) FROM temp GROUP BY userQuery) ORDER BY sessionId DESC;")
with_dateDF = userQuery.withColumn('date', date_format('timestamp', 'yyyy-MM-dd'))
with_dateDF.toPandas().to_csv("query_files.csv", header=True)

#write entire data to csv
#date_inclusive.toPandas().to_csv("sample_files.csv", header=True)

#date_inclusive.show(10)
#filter rows using date...last 100 days 
#filter_date = date_inclusive.filter(F.col('date') <= F.date_sub(F.current_date(), 100))
#filter_date.show(250)

#q_time.show()

#getting the last row of each sessionID
feat3 = my_spark.sql("SELECT sessionId,timestamp, userQuery, event, flowName, intentName FROM temp WHERE id IN (SELECT MAX(id) FROM temp GROUP BY sessionId) ORDER BY sessionId DESC;")
feat3.createOrReplaceTempView("tabl")
groupDF = my_spark.sql("SELECT sessionId,timestamp, userQuery, event,flowName,intentName FROM tabl")
#groupDF = my_spark.sql("SELECT flowName, count(*) as N FROM tabl GROUP BY flowName")
q2_time = groupDF.withColumn('time', date_format('timestamp', 'HH:mm:ss'))
date_inclusive2 = q2_time.withColumn('date', date_format('timestamp', 'yyyy-MM-dd'))
#write entire data to csv
date_inclusive2.toPandas().to_csv("analysis_data.csv", header=True)
#writing df to pandas
pd_counts = date_inclusive2.toPandas()
pd_counts.head()





,sessionId,timestamp,userQuery,event,flowName,intentName,time,date
0,zzvep9d4jxq3fvwxc2qtp2d5hjzppece,2021-01-17 15:17:58.520,,policy_details,policy_details,None,15:17:58,2021-01-17
1,zzlzdxcog9mah27rlf0txl51soofdolv,2021-01-18 13:03:51.475,,Policy_4DigitVerify,Policy_4DigitVerify,None,13:03:51,2021-01-18
2,zzkhhe1ipmm1objrw4r7ln3okw6toc9e,2021-01-28 14:52:30.643,,policy_details,policy_details,None,14:52:30,2021-01-28
3,zzfllkihdcufgy8w9z7v7rdz9e4miekd,2021-01-13 18:28:21.589,,policy_details,policy_details,None,18:28:21,2021-01-13
4,zzchk2s5uk6jvkwne4fe7lfzrevl3n2y,2021-01-01 14:19:53.556,Hello.,None,policy_details,None,14:19:53,2021-01-01


In [68]:
fd = pd_counts
start_date = "2021-01-18 13:03:51.475"
end_date = datetime.now()
print(end_date)

edate = datetime.strptime("2021-02-01T14:00:00.000-08:00".split(".")[0],'%Y-%m-%dT%H:%M:%S').strftime("%m-%d-%Y %H:%M:%S.%f")
print(edate)
#still need to check whether response is a date
#filter data using date if day,days months years in query
fd_filtered = fd.filter_date("timestamp", start_date, end_date)
print(fd_filtered)

2021-04-19 02:05:19.608766
02-01-2021 14:00:00.000000
                             sessionId               timestamp  \
1     zzlzdxcog9mah27rlf0txl51soofdolv 2021-01-18 13:03:51.475   
2     zzkhhe1ipmm1objrw4r7ln3okw6toc9e 2021-01-28 14:52:30.643   
6     zzc4akry8mjjzmysziv7u48uyib8ucfk 2021-01-19 15:11:48.051   
7     zz9j56kd780bq8szgelcyhaehmtf6hri 2021-02-06 18:42:33.942   
10    zyt5mo3a9kj1ilwh332jgqgo04l71awh 2021-01-20 13:35:30.698   
...                                ...                     ...   
7025  0174hj95m19tpc0rgff75f92k6ijwilu 2021-01-29 20:55:57.765   
7026  00vgaemels355aa6c43hsn3maa1hdqtk 2021-01-18 14:47:08.042   
7027  00oesptpfdz2qmqeu3i0zmzkkm9xwebz 2021-01-23 14:43:57.581   
7030  00cma960xlqioes5iz36hbmnbidupj6q 2021-02-06 18:27:07.927   
7032  008zu4maqohthhl4yaqwnx2xhtk4dwqk 2021-01-19 17:05:02.586   

                                     userQuery                event  \
1                                               Policy_4DigitVerify   
2          

In [22]:
pd_counts = date_inclusive2.toPandas()
f = pd_counts.count()
Total_number_of_calls = f["sessionId"]
#pd_countss = ps.sqldf("SELECT flowName, count(*) as N FROM pd_counts GROUP BY flowName")
#print(pd_countss.head())

In [ ]:
#total number of calls
import pandas as pd
unique_df = pd.read_csv("analysis_data.csv")
#unique_df.head()
#unique_df["sessionId"].count
f = unique_df.count()
print(f["sessionId"])
#determine failed and succesful calls
g = unique_df.set_index(["flowName", "sessionId"]).count(level="flowName")
print(g)

In [2]:
#total number of calls
import pandas as pd
unique_df = pd.read_csv("analysis_data.csv")
#unique_df.head()
#unique_df["sessionId"].count
f = unique_df.count()
print(f["sessionId"])
#determine failed and succesful calls


7033


In [69]:

def big_function(message):
    exl_date_entities = ["date","time"]
    def extract_value(match):
        if match["value"].get("type") == "interval":
            value = {"to": match["value"].get("to", {}).get("value"),
                     "from": match["value"].get("from", {}).get("value")}
        else:
            value = match["value"].get("value")

        return value

    def filter_irrelevant_matches(matches, requested_dimensions):
        """Only return dimensions the user configured"""

        if requested_dimensions:
            return [match
                    for match in matches
                    if match["dim"] in requested_dimensions]
        else:
            return matches


    def convert_duckling_format_to_rasa(matches):
        extracted = []

        for match in matches:
            value = extract_value(match)
            entity = {"start": match["start"],
                      "end": match["end"],
                      "text": match.get("body", match.get("text", None)),
                      "value": value,
                      "confidence": 1.0,
                      "additional_info": match["value"],
                      "entity": match["dim"]}

            extracted.append(entity)

        return extracted

    def convert_duckling_date_format_to_exl(extracted):

        entity_name = "date"
        modified = []
        for i in extracted:
            if i["entity"] == "time":
                if type(i["value"]) is dict:
                    if i["value"]["from"]:
                        sdate = datetime.strptime(i["value"]["from"].split(".")[0],
                                                  '%Y-%m-%dT%H:%M:%S').strftime("%m-%d-%Y %H:%M:%S.%f")
                        modified.append({"entity": entity_name,
                            "value":sdate})
                    if i["value"]["to"]:
                        edate = datetime.strptime(i["value"]["to"].split(".")[0],
                           '%Y-%m-%dT%H:%M:%S').strftime("%m-%d-%Y %H:%M:%S.%f")
                        modified.append({"entity": entity_name,
                            "value":edate})
                else:
                    if i["value"]:
                        print(i["value"].split("T")[0])
                        date = datetime.strptime(i["value"].split(".")[0],
                            '%Y-%m-%dT%H:%M:%S').strftime("%m-%d-%Y %H:%M:%S.%f")
                        modified.append({"entity": entity_name,
                            "value":date})
            
            else:
                modified.append(i)
        return modified

    component_config = {
        # by default all dimensions recognized by duckling are returned
        # dimensions can be configured to contain an array of strings
        # with the names of the dimensions to filter for
        "dimensions": None,

        # http url of the running duckling server
        "url": None,

        # locale - if not set, we will use the language of the model
        "locale": None,

        # timezone like Europe/Berlin
        # if not set the default timezone of Duckling is going to be used
        "timezone": None
    }

    def _locale():
        if not component_config.get("locale"):
            # this is king of a quick fix to generate a proper locale
            # works most of the time
            language = "en"
            locale_fix = "{}_{}".format(language,language.upper())
            component_config["locale"] = locale_fix
        return component_config.get("locale")

    def _url():
        url = "http://localhost:8000"
        return url

    def _payload(text, reference_time):
        return {
            "text": text,
            "locale": _locale(),
            "tz": component_config.get("timezone"),
            "reftime": reference_time
        }

    def _duckling_parse(text, reference_time):
        """Sends the request to the duckling server and parses the result."""
        #text = text.replace("this summer","december")
        #text = text.replace("flower","december")
        #print("text is.....",text)
        try:
            payload = _payload(text, reference_time)
            headers = {"Content-Type": "application/x-www-form-urlencoded; "
                                       "charset=UTF-8"}
            response = requests.post(_url() + "/parse",
                                     data=payload,
                                     headers=headers)
            if response.status_code == 200:
                return simplejson.loads(response.text)
            else:
                logger.error("Failed to get a proper response from remote "
                             "duckling. Status Code: {}. Response: {}"
                             "".format(response.status_code, response.text))
                return []
        except requests.exceptions.ConnectionError as e:
            logger.error("Failed to connect to duckling http server. Make sure "
                         "the duckling server is running and the proper host "
                         "and port are set in the configuration. More "
                         "information on how to run the server can be found on "
                         "github: "
                         "https://github.com/facebook/duckling#quickstart "
                         "Error: {}".format(e))
            return []

    #@staticmethod
    def _reference_time_from_message(message):
        if message.time is not None:
            try:
                return int(message.time) * 1000
            except ValueError as e:
                logging.warning("Could not parse timestamp {}. Instead "
                                "current UTC time will be passed to "
                                "duckling. Error: {}".format(message.time, e))
        # fallbacks to current time, multiplied by 1000 because duckling
        # requires the reftime in miliseconds
        return int(time.time()) * 1000

    if _url() is not None:
        #message = "yesterday"
        #reference_time = _reference_time_from_message(message)
        # function to convert words to numbers
        reference_time = None
        msg = message.replace(".","")
        #print("msg is ..............",msg)
        #message_text = self.convert_message_text(mesg)
        #print("the message text is.......",message_text)
        matches = _duckling_parse(msg, reference_time)
        #print("matches ..................",matches)
        dimensions = component_config["dimensions"]
        relevant_matches = filter_irrelevant_matches(matches, dimensions)
        print(relevant_matches)
        extracted = convert_duckling_format_to_rasa(relevant_matches)
        #print("extracted..........................",extracted)
        dextracted = convert_duckling_date_format_to_exl(extracted)
        print(dextracted)
    else:
        dextracted = []
        logger.warning("Duckling HTTP component in pipeline, but no "
                       "`url` configuration in the config "
                       "file nor is `RASA_DUCKLING_HTTP_URL` "
                       "set as an environment variable.")
    return dextracted

big_function("2pm yesterday")



[{'body': '2pm yesterday', 'start': 0, 'value': {'values': [{'value': '2021-04-17T14:00:00.000-07:00', 'grain': 'hour', 'type': 'value'}], 'value': '2021-04-17T14:00:00.000-07:00', 'grain': 'hour', 'type': 'value'}, 'end': 13, 'dim': 'time', 'latent': False}]
2021-04-17
[{'entity': 'date', 'value': '04-17-2021 14:00:00.000000'}]


[{'entity': 'date', 'value': '04-17-2021 14:00:00.000000'}]

In [35]:
pd_c = pd_countss
testing = pd_countss["flowName"]
testing2 = pd_countss["N"]
title = [ each for each in testing]
counts = [ each for each in testing2]

if "generateClaimNumber" in title:
    print("yea")
    indx = title.index("generateClaimNumber")
    TKU_indx = title.index("Thank You FLow")
    number_tku = counts[TKU_indx]
    number = counts[indx]
    num = number + number_tku
    pd_c.loc[-1] = ["Successful calls",num]
    pd_c.index = pd_c.index + 1  # shifting index
    pd_c = pd_c.sort_index()
        
else:
    print("no")

yea


In [103]:
pd_c.to_csv("count_data.csv", header=True)

In [90]:
import os
from nl4dv import NL4DV
import time
import logging
import os
import requests
from datetime import datetime
import simplejson
def number_vis(message,pd_countss):
    #print(pd_countss)
    #determine failed and succesful calls
    try:
        pd_countsss = pd_countss.dropna()
        title = [ each for each in pd_countsss['flowName']]
        counts = [ each for each in pd_countsss['N']]
        if "generateClaimNumber" in title:
            print("yea")
            indx = title.index("generateClaimNumber")
            TKU_indx = title.index("Thank You FLow")
            number_tku = counts[TKU_indx]
            number = counts[indx]
            num = number + number_tku
            pd_countsss.loc[-1] = ["Successful calls",num]
            pd_countsss.index = pd_countsss.index + 1  # shifting index
            pd_countsss = pd_countsss.sort_index()

        else:
            print("no")

        text =  [ each for each in pd_countsss['flowName']]
        counts = [ each for each in pd_countsss['N']]
        def remove_noise_punct(text):
            #text = text.translate(str.maketrans('', '', punctuation))
            for char in punctuation:
                text = text.replace(char, ' ')
            return text
        textss = []
        for i in text:
            if i == "agenttransfer_flow":
                i = "agent transfer flow"
            else:
                i = i
            #print(i)
            r = remove_noise_punct(i)
            r =  re.sub(r"(\w)([A-Z])", r"\1 \2", r)
            textss.append(r)
        print(textss)
        num = []
        for i in textss:
            score = fuzz.partial_token_sort_ratio(message,i)
            num.append(score)
        maxi = max(num)
        max_index = num.index(maxi)
        numbers = counts[max_index]   
        #print(textss[max_index])
        #print(num)
        #print(textss)
        if "how many" in message:
            #resp = message.strip("how many")
            resps = message.split("how many")
            resp = resps[1]
            result = str(numbers) +" "+ resp
            #print(result)
            #print("nnnn")
        else:

            if "calls" in message:
                result = str(numbers) + " "+"calls"
            else:
                result = str(numbers) + " in "+ textss[max_index]
                result = result
            #print(result)
    except:
        result = "I don't have records on the requested date"
        maxi = 0
        pass
    return result,maxi
#    pass

In [82]:
def word_cloud():
    vis = VegaLite({
      "$schema": "https://vega.github.io/schema/vega/v5.json",
      "description": "A word cloud visualization of the user query.",
      "width": 800,
      "height": 400,
      "padding": 0,

      "data": [
        {
          "name": "table",
          "url": "data1.json",
          "transform": [
            {
              "type": "countpattern",
              "field": "data",
              "case": "upper",
              "pattern": "[\\w']{3,}",
              "stopwords": "(i|me|my|myself|we|us|our|ours|ourselves|you|your|yours|p.m|A.M|P.M|yourself|yourselves|he|him|his|himself|she|her|hers|herself|it|its|itself|they|them|their|theirs|themselves|what|which|who|whom|whose|this|that|these|those|am|is|are|was|were|be|been|being|have|has|had|having|do|does|did|doing|will|would|should|can|could|ought|i'm|you're|he's|she's|it's|we're|they're|i've|you've|we've|they've|i'd|you'd|he'd|she'd|we'd|they'd|i'll|you'll|he'll|she'll|we'll|they'll|isn't|aren't|wasn't|weren't|hasn't|haven't|hadn't|doesn't|don't|didn't|won't|wouldn't|shan't|shouldn't|can't|cannot|couldn't|mustn't|let's|that's|who's|what's|here's|there's|when's|where's|why's|how's|a|an|the|and|but|if|or|because|as|until|while|of|at|by|for|with|about|against|between|into|through|during|before|after|above|below|to|from|up|upon|down|in|out|on|off|over|under|again|further|then|once|here|there|when|where|why|how|all|any|both|each|few|more|most|other|some|such|no|nor|not|only|own|same|so|than|too|very|say|says|said|shall)"
            },
            {
              "type": "formula", "as": "angle",
              "expr": "[-45, 0, 45][~~(random() * 3)]"
            },
            {
              "type": "formula", "as": "weight",
              "expr": "if(datum.text=='VEGA', 600, 300)"
            }
          ]
        }
      ],

      "scales": [
        {
          "name": "color",
          "type": "ordinal",
          "domain": {"data": "table", "field": "text"},
          "range": ["#d5a928", "#652c90", "#939597"]
        }
      ],

      "marks": [
        {
          "type": "text",
          "from": {"data": "table"},
          "encode": {
            "enter": {
              "text": {"field": "text"},
              "align": {"value": "center"},
              "baseline": {"value": "alphabetic"},
              "fill": {"scale": "color", "field": "text"}
            },
            "update": {
              "fillOpacity": {"value": 1}
            },
            "hover": {
              "fillOpacity": {"value": 0.5}
            }
          },
          "transform": [
            {
              "type": "wordcloud",
              "size": [800, 400],
              "text": {"field": "text"},
              "rotate": {"field": "datum.angle"},
              "font": "Helvetica Neue, Arial",
              "fontSize": {"field": "datum.count"},
              "fontWeight": {"field": "datum.weight"},
              "fontSizeRange": [12, 56],
              "padding": 2
            }
          ]
        }
      ]
    })
    return vis

In [91]:
import janitor
import os
import json
import pandas as pd
import pandasql as ps
from datetime import date
from string import punctuation
from vega import VegaLite
from nl4dv import NL4DV
from fuzzywuzzy import fuzz
import re
parser = "/home/tushar/spark/stanford-parser-full-2018-10-17/stanford-parser.jar"
model =  "/home/tushar/spark/stanford-parser-full-2018-10-17/stanford-parser-3.9.2-models.jar"

def main_function(message):
    message = str(message)
    filenames = "analysis_data.csv" 
    rd = pd.read_csv(filenames)
    file = "query_files.csv"
    rd2 = pd.read_csv(file)
    pd_countss = ps.sqldf("SELECT flowName, count(*) as N FROM pd_counts GROUP BY flowName")
    f = rd.count()
    print(f["sessionId"])
    Total_number = f["sessionId"]
    pd_countss.loc[-1] = ["Total number",Total_number]
    pd_countss.index = pd_countss.index + 1  # shifting index
    pd_countss = pd_countss.sort_index()
    #print(pd_countss.head())
    try:
        date_function= big_function(message)
        start_date = date_function[0]['value']
        end_date = datetime.now()
        #still need to check whether response is a date
        #filter data using date if day,days months years in query
        df_filtered = rd.filter_date("date", start_date, end_date)
        #print(df_filtered)
        df_filtered2 = rd2.filter_date("date", start_date, end_date)
        file = "filtered_text.csv"
        df_filtered2.to_csv(file,header = True)

        #write to csv 
        filenames = "filtered_data.csv"
        df_filtered.to_csv(filenames,header = True)
        pd_countss = ps.sqldf("SELECT flowName, count(*) as N FROM df_filtered GROUP BY flowName")
        f = df_filtered.count()
        print(f["sessionId"])
        Total_number = f["sessionId"]
        pd_countss.loc[-1] = ["Total number",Total_number]
        pd_countss.index = pd_countss.index + 1  # shifting index
        pd_countss = pd_countss.sort_index()
        
        #print(pd_countss)
        print("completed")
    except:
        pass
    
    pd_countss = pd_countss
    #print(pd_countss)
    response,threshold = number_vis(message, pd_countss)
    print(threshold)

    if "create" in message and threshold <= 65:
            
        #create .json synonym list
        #change the nl4dv filename to created csv

        data_url=filenames
        print(data_url)
        alias_url2 = "nl4dv-master/examples/assets/aliases/data_alias.json"
        # -------------------- Special attributes / tokens ---------------------------
        label_attribute = "Title"
        ignore_words = ['movie']

        # -------------------- Attribute Datatype Overrides ---------------------------
        attribute_datatypes = {"Release Year": "T"}
        # Initialize NL4DV and set the above configurations
        dependency_parser_config = {'name': 'corenlp','model': model,'parser': parser}
        nl4dv_instance = NL4DV(verbose=False)
        nl4dv_instance.set_data(data_url=data_url)
        nl4dv_instance.set_alias_map(alias_url=alias_url2)
        nl4dv_instance.set_label_attribute(label_attribute=label_attribute)
        nl4dv_instance.set_ignore_words(ignore_words=ignore_words)
        #nl4dv_instance.set_attribute_datatype(attr_type_obj=attribute_datatypes)
        nl4dv_instance.set_dependency_parser(config=dependency_parser_config)
        query = message
        print(query)
        # -------------------- Ask Query ---------------------------
        #nl4dv_response = nl4dv_instance.analyze_query(query, debug=True)
        #print("\nAttributes List:")
        #print(nl4dv_response['attributeMap'].keys())
        response = nl4dv_instance.render_vis(query)
    elif "word" in message:
        print(message)
        #file = "query_files.csv"
        #try:
        print(file)
        data = pd.read_csv(file)
        query = data['userQuery']

        q = query.to_string(index = False)
        if query.empty == True:
            response = " I don't have records on the requested date"
        else:
            r = q.replace("NaN","")

            res = " ".join(r.split())
            #f = r.replace(" ","")
            texts = str(res)
            text_without_number =  ''.join(filter(lambda item: not item.isdigit(), texts))

            import json
            with open('data1.json', 'w') as outfile:
                json.dump(text_without_number, outfile)
            print("done")
            response = word_cloud()
            #except:
            #
            response = response
        
    return response





In [101]:
main_function("what is the total number of calls transfer to agent ")
#how many calls got canceled at email address
#show wordcloud of user query in last 2 months
#how many calls was transfered to agent since 29th of december 2020

7033
[]
[]
yea
['Successful calls', 'Total number', 'Caller Details', 'Garage Name', 'Insured name flow', 'Insured name no intent', 'Insured name yes intent', 'Policy 4 Digit Verify', 'Thank You F Low', 'accident City', 'accident Date', 'accident Date no', 'accident Date yes', 'accident Description', 'accident Time', 'agent transfer flow', 'ask email', 'check rmn', 'driver Name', 'garage City yes', 'garage Pincode yes', 'garage city', 'garage pincode', 'generate Claim Number', 'insured Email no', 'insured Email yes', 'insured Number no', 'insured Number yes', 'insured email', 'insured number', 'insured number verify', 'intimator number', 'policy details', 'vehicle Number no', 'vehicle Number yes', 'vehicle number', 'weather', 'welcome', 'workshop details', 'workshop no', 'workshop yes']
75


'7033 calls'

## 

In [259]:
messe = "what was transfered to word cloud agent since 29th of december 2020"
if "word"  in messe:
    print("yeah")
else:
    print("no")

yeah


In [117]:
message = "can you tell me how many calls got canceled at email address"
number_vis(message)


TypeError: number_vis() missing 1 required positional argument: 'pd_countss'